In [104]:
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import porter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

In [6]:
true_df = pickle.load(open("pickle_files/true_df.pkl", "rb" ))
true_df_backup = pickle.load(open("pickle_files/true_df_backup.pkl", "rb" ))

In [28]:
words=pos_tag(word_tokenize("Who's going to that thing today?"))
for w in words:
    print(w[1])

WP
VBZ
VBG
TO
DT
NN
NN
.


In [145]:
stemmer = porter.PorterStemmer()
stopwords = stopwords.words()

In [146]:
def clean_text(text):
    '''Removes stop words and changes word to stem words'''
    cleaned_text = []
    for post in text:
        cleaned_words = []
        for word in post.split():
            low_word = stemmer.stem(word.lower())
            if low_word not in stopwords:
                cleaned_words.append(low_word)
        cleaned_text.append(' '.join(cleaned_words))
    return cleaned_text

In [147]:
cleaned_text = clean_text(true_df.body)

In [55]:
'''with open('clean_text.pkl', 'wb') as picklefile:
        pickle.dump(clean_text, picklefile)'''

In [56]:
'''clean_text = pickle.load(open("pickle_files/clean_text.pkl", "rb" ))'''


In [164]:
cleaned_text[:5]

['follow statement post verifi twitter account u.s. presid donald trump, @realdonaldtrump @potus. u.s. presid donald trump hold make america great ralli nashvil municip auditorium nashville, tennessee, u.s., may 29, 2018. reuters/leah millisth opinion express hi own. reuter edit statement confirm accuracy. @realdonaldtrump : - mitch mcconnel announc cancel senate’ august recess. great, mayb democrat final get someth done accept high crime high taxes. need border security! [0002 edt] - great night republicans! congratul john cox realli big number california. win. even fake news cnn said trump impact wa realli big, much bigger ever thought possible. much big blue wave, may big red wave. work hard! [0916 edt] - gold star father, ceejay metcalf, whose great michael wa honor white house, wa fantast morn @foxandfriends. special man! [0937 edt] - fake news media unfair, vicious, wife great first lady, melania. dure recoveri surgeri report everyth near death, facelift, left w.h. (and me) n.y. 

In [186]:
def drop_nouns(text_list):
    no_nouns = []
    all_nouns = ['NN' or 'NNS' or 'NNP' or 'NNPS']
    for x in range(len(text_list)):
        words=pos_tag(word_tokenize(cleaned_text[x]))
        werdz = [s for s in words if s[-1] != 'NN']# or 'NNS' or 'NNP' or 'NNPS']
        werdz = [s for s in werdz if s[-1] != 'NNS']
        werdz = [s for s in werdz if s[-1] != ',']
        werdz = [s for s in werdz if s[-1] != '.']
        werdz = [s for s in werdz if s[-1] != ':']
        werdz = [s for s in werdz if s[-1] != 'CD']
        werdz = [s for s in werdz if s[-1] != '(']
        werdz = [s for s in werdz if s[-1] != ')']
        
        no_nouns.append(werdz)
    return no_nouns

In [189]:
text_w_no_nouns = drop_nouns(cleaned_text)

In [190]:
short_text = []
for text in text_w_no_nouns:
    for part in text:
        short_text.append(part[0])

In [191]:
count_vectorizer2 = CountVectorizer(ngram_range=(1, 4),  
                                   stop_words='english', 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)

X = count_vectorizer2.fit_transform(short_text)

In [ ]:
n_topics = 20
n_iter = 10
lda = LatentDirichletAllocation(n_topics=n_topics,
                                max_iter=n_iter,
                                random_state=42,
                               learning_method='online')
X_centered_projected = lda.fit_transform(X)

/Users/NickThomas/Desktop/anaconda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [ ]:
def display_topics(model, feature_names, no_top_words):
    '''Creates a list of words in each topics'''
    for ix, topic in enumerate(model.components_):
        print("Topic ", ix+1)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
display_topics(lda,count_vectorizer2.get_feature_names(),20)

In [ ]:
with open('X_centered_projected.pkl', 'wb') as picklefile:
        pickle.dump(X_centered_projected, picklefile)

In [ ]:
"""with open('quick_to_the_batmobile.pkl', 'wb') as picklefile:
        pickle.dump(quick_to_the_batmobile, picklefile)"""

In [ ]:
X_centered_projected_w_nouns = pickle.load(open("pickle_files/X_centered_projected_w_nouns", "rb" ))